In [1]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import PIL.Image as Image
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights

In [2]:
classes = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]

In [3]:
class ResNet50(nn.Module):
    def __init__(self, num_classes, weights=None):
        super(ResNet50, self).__init__()
        # Charger le modèle pré-entraîné ResNet50
        self.resnet = resnet50(pretrained=True if weights is None else False)

        # Extraire les couches sauf la couche de classification finale (dernière couche)
        self.features = nn.Sequential(*list(self.resnet.children())[:-1])

        # Ajouter une nouvelle couche de classification adaptée au nombre de classes (26 lettres)
        self.fc = nn.Linear(self.resnet.fc.in_features, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print("Using the ASL DATASET")

Using device: cuda
Using the ASL DATASET


In [5]:
model = ResNet50(26).to(device)
model.load_state_dict(torch.load('pytorch_model_Thomas.pth', map_location=device))

C:\Users\Lorenzo\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Lorenzo\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [6]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

    transform = transforms.Compose([
        transforms.CenterCrop(min(image.size)),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))  # Normalisation des valeurs de pixel centré autour de 0 (-1 ; 1)
    ])

In [7]:
def classify(model, transform, image_path, classes):
    model = model.eval()
    image = Image.open(image_path)  # Ouvrir image
    
    image = transform(image).float()  # Transformer image
    image = image.unsqueeze(0)

    # Assurez-vous que les données d'entrée sont sur le même dispositif que le modèle
    image = image.to(device)

    output = model(image)
    _, predicted = torch.max(output.data, 1)

    print(classes[predicted.item()])

In [8]:
classify(model, transform, "./Sign/L2.jpg", classes)

NameError: name 'transform' is not defined